# Using tf.data

https://www.tensorflow.org/programmers_guide/datasets

In [1]:
from PIL import Image
import numpy as np
import tensorflow as tf

/Users/tjw/anaconda3/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dataset = tf.data.Dataset.range(10)
print(dataset.output_types)
print(dataset.output_shapes)

iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()
with tf.Session() as sess:
    for i in range(10):
        print(sess.run(next_element))

<dtype: 'int64'>
()
0
1
2
3
4
5
6
7
8
9


### make_initializable_iterator

In [3]:
tf.reset_default_graph()
min_value = tf.placeholder(tf.int64, shape=[])
max_value = tf.placeholder(tf.int64, shape=[])
dataset = tf.data.Dataset.range(min_value, max_value)
iterator = dataset.make_initializable_iterator()
next_element = iterator.get_next()

with tf.Session() as sess:
    # Initialize an iterator over a dataset with 10 elements.
    sess.run(iterator.initializer, feed_dict={min_value: 0, max_value: 5})
    for i in range(5):
        print(sess.run(next_element))

    # Initialize the same iterator over a dataset with 10 elements.
    sess.run(iterator.initializer, feed_dict={min_value: 100, max_value: 105})
    for i in range(5):
        value = sess.run(next_element)
        print(value)

0
1
2
3
4
100
101
102
103
104


## from_tensor_slices

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(tf.random_uniform([10, 3]))

print(dataset.output_types)
print(dataset.output_shapes)

iterator = dataset.make_initializable_iterator()

next_element = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(10):
        print(sess.run(next_element))

<dtype: 'float32'>
(3,)
[0.38122594 0.8570912  0.5940461 ]
[0.34225667 0.37440777 0.9416013 ]
[0.2731502  0.32971644 0.00413644]
[0.20123243 0.8915731  0.24754727]
[0.59967387 0.5907049  0.81413865]
[0.4130001  0.17984414 0.42022014]
[0.7442223  0.7332072  0.64030683]
[0.35583568 0.12731469 0.67755544]
[0.22931314 0.69057596 0.56508124]
[0.44057524 0.48460376 0.30375147]


## Q
計算 $1+2+...+10$

### Reinitializable (one interator with different datasets)

In [5]:
# Define training and validation datasets with the same structure.
training_dataset = tf.data.Dataset.range(10).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64))
validation_dataset = tf.data.Dataset.range(5)

# two dataset are compatible
assert training_dataset.output_types == validation_dataset.output_types
assert training_dataset.output_shapes == validation_dataset.output_shapes

iterator = tf.data.Iterator.from_structure(training_dataset.output_types,
                                           training_dataset.output_shapes)
next_element = iterator.get_next()

training_init_op = iterator.make_initializer(training_dataset)
validation_init_op = iterator.make_initializer(validation_dataset)

def loop_through_dataset(ds_name, n):
        for _ in range(n):
            print(ds_name, _, sess.run(next_element))

with tf.Session() as sess:
    for epoch in range(3):    
        print("epoch", epoch)
        # training
        sess.run(training_init_op)
        loop_through_dataset("train", 10)

        # Validation
        sess.run(validation_init_op)
        loop_through_dataset("validation", 5)


epoch 0
train 0 5
train 1 -1
train 2 -1
train 3 7
train 4 3
train 5 9
train 6 6
train 7 16
train 8 6
train 9 14
validation 0 0
validation 1 1
validation 2 2
validation 3 3
validation 4 4
epoch 1
train 0 3
train 1 8
train 2 -1
train 3 1
train 4 10
train 5 -5
train 6 9
train 7 3
train 8 14
train 9 4
validation 0 0
validation 1 1
validation 2 2
validation 3 3
validation 4 4
epoch 2
train 0 3
train 1 2
train 2 -3
train 3 8
train 4 2
train 5 14
train 6 13
train 7 -2
train 8 15
train 9 1
validation 0 0
validation 1 1
validation 2 2
validation 3 3
validation 4 4


## MNIST Dataset

In [6]:
import lzma
import pickle
with lzma.open("mnist.pkl.xz", 'rb') as f:
    train_set, validation_set, test_set = pickle.load(f, encoding='latin1')

In [7]:
train_X, train_y = train_set
validation_X, validation_y = validation_set
test_X, test_y = test_set
train_Y = np.eye(10)[train_y]
test_Y = np.eye(10)[test_y]
validation_Y = np.eye(10)[validation_y]

In [8]:
from IPython.display import display
def showX(X):
    int_X = (X*255).clip(0,255).astype('uint8')
    # N*784 -> N*28*28 -> 28*N*28 -> 28 * 28N
    int_X_reshape = int_X.reshape(-1,28,28).swapaxes(0,1).reshape(28,-1)
    display(Image.fromarray(int_X_reshape))
# 訓練資料， X 的前 20 筆
showX(train_X[:20])
print(train_y)

[5 0 4 ... 8 4 8]


In [9]:
train_data = tf.data.Dataset.from_tensor_slices((train_X, train_Y))

iterator = train_data.batch(4).make_initializable_iterator()

next_minibatch = iterator.get_next()
with tf.Session() as sess:
    sess.run(iterator.initializer)
    for i in range(3):
        print(sess.run(next_minibatch)[1])

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]]
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]


# Multilayer Convolutional Network

In [10]:
iterator = train_data.batch(32).make_initializable_iterator()
X, Y_ = iterator.get_next()

In [11]:
# 設定 weight 和 bais
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name ='W')
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name = 'b')

# 設定 cnn 的 layers
def conv2d(X, W):
    return tf.nn.conv2d(X, W, strides=[1,1,1,1], padding='SAME')
def max_pool_2x2(X):
    return tf.nn.max_pool(X, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

# fisrt layer
with tf.name_scope('conv1'):
    ## variables
    W_conv1 = weight_variable([3,3,1,32])
    b_conv1 = bias_variable([32])
    ## build the layer
    X_image = tf.reshape(X, [-1, 28, 28, 1])
    h_conv1 = tf.nn.relu(conv2d(X_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

# second layer
with tf.name_scope('conv2'):
    ## variables
    W_conv2 = weight_variable([3,3,32,64])
    b_conv2 = bias_variable([64])
    ## build the layer
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
# fully-connected layer
with tf.name_scope('full'):
    W_fc1 = weight_variable([7*7*64, 1024])
    b_fc1 = bias_variable([1024])
    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
    
# Dropout:  A Simple Way to Prevent Neural Networks from Over fitting
# https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf
with tf.name_scope('dropout'):
    keep_prob = tf.placeholder("float", name="keep_prob")
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

# Readout
with tf.name_scope('readout'):
    W_fc2 = weight_variable([1024,10])
    b_fc2 = bias_variable([10])
    Y = tf.matmul(h_fc1_drop, W_fc2)+b_fc2
    
cross_entropy =  tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_, logits=Y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
prediction = tf.argmax(Y, 1, name="prediction")
correct_prediction = tf.equal(prediction, tf.argmax(Y_, 1), name="correction")
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(iterator.initializer)
for i in range(50):
    print(i)
    sess.run(train_step, {keep_prob: 0.5 })

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27